In [1]:
!nvidia-smi

Mon Jun 13 10:30:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 9.6 MB/s 
     |████████████████████████████████| 4.2 MB 71.9 MB/s 
     |████████████████████████████████| 1.1 MB 55.2 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
     |████████████████████████████████| 140 kB 80.4 MB/s 
     |████████████████████████████████| 212 kB 103.7 MB/s 
     |████████████████████████████████| 86 kB 7.6 MB/s 
     |████████████████████████████████| 596 kB 95.3 MB/s 
     |████████████████████████████████| 127 kB 100.4 MB/s 
     |████████████████████████████████| 6.6 MB 98.3 MB/s 
     |████████████████████████████████| 144 kB 80.8 MB/s 
     |████████████████████████████████| 271 kB 79.2 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
     |████████████████████████████████| 112 kB 65.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalli

In [4]:
import argparse
import json
import random

import torch
from datasets import load_dataset
from tqdm import tqdm
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    BlenderbotForConditionalGeneration,
    BlenderbotTokenizer,
)

In [5]:
def preprocess(example):

    example["personas"] = [f"your persona: {p}" for p in example["personas"]]
    example["context"] = "\n".join(
        example["personas"]
        + (["additional_context"] if example["additional_context"] else [])
        + example["previous_utterance"]
    )

    return example

In [21]:
dataset = load_dataset("blended_skill_talk", split='train')
dataset = dataset.map(
    preprocess,
    remove_columns=[
        "free_messages",
        "guided_messages",
        "suggestions",
        "personas",
        "additional_context",
        "previous_utterance",
    ],
)
dataset = dataset.select(random.sample(range(len(dataset)), 200))

Using custom data configuration default
Reusing dataset blended_skill_talk (/root/.cache/huggingface/datasets/blended_skill_talk/default/1.0.0/bded69fdeee98ed8bba2ef088ac9dfd74e9ad0b95b1de5d51e333cee6f6261aa)
Loading cached processed dataset at /root/.cache/huggingface/datasets/blended_skill_talk/default/1.0.0/bded69fdeee98ed8bba2ef088ac9dfd74e9ad0b95b1de5d51e333cee6f6261aa/cache-0da07bcaedcc65cb.arrow


In [22]:
print(len(dataset))
#dataset['context']

200


In [8]:
test_set = load_dataset("blended_skill_talk", split='test')
test_set = test_set.map(
    preprocess,
    remove_columns=[
        "free_messages",
        "guided_messages",
        "suggestions",
        "personas",
        "additional_context",
        "previous_utterance",
    ],
)
print(len(test_set))

Using custom data configuration default
Reusing dataset blended_skill_talk (/root/.cache/huggingface/datasets/blended_skill_talk/default/1.0.0/bded69fdeee98ed8bba2ef088ac9dfd74e9ad0b95b1de5d51e333cee6f6261aa)


  0%|          | 0/980 [00:00<?, ?ex/s]

980


In [9]:
print(test_set[0:10]['context'])

["your persona: i hate talking to people.\nyour persona: i believe dragons are real.\nadditional_context\nWow, I am never shy. Do you have anxiety?\nYes. I end up sweating and blushing and feel like i'm going to throw up.", 'your persona: i have three daughters.\nyour persona: my wife and i like to go scuba diving.\nMy turtle ran away from me today.\nOh my god. Did you guys have a fight?', 'your persona: i hate the taste of fish.\nyour persona: i like to paint.\nOur son in the Army is taking a leave to visit us next month.\nOh, you must be so excited! I have a son in the Army as well and I love when he gets to visit! ', "your persona: my favorite movie is good burger.\nyour persona: i like canning and whittling.\nthat's awesome , i like running in the morning instead of night\nsounds nice . i only eat meat even though i enjoy canning", 'your persona: my hair is black.\nyour persona: i like rock climbing.\nadditional_context\nAre there different skill levels? \nI do not know of specific

In [10]:
import spacy
with open("/content/drive/MyDrive/adl-final/final_project_scripts/keywords.json") as f:
    keywords = json.load(f)

nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    # lemmatize words in keywords
for key, val in keywords.items():
    # separate words by its length (one, others)
    one_lemma = []
    multi_lemma = []
    for word in val:
        split = [token.lemma_ for token in nlp(word)]
        if len(split) >= 2:
            multi_lemma.append(" ".join(split))
        else:
            one_lemma.append(split[0])
        keywords[key] = [one_lemma, multi_lemma]

def hit(text):
    lemma_utterance = [token.lemma_ for token in nlp(text)]
    cnt = 0
    for key, (one, multi) in keywords.items():
        intersection = set(one) & set(lemma_utterance)
        # check whether the word, the length is bigger than 2, is in the utterance
        for m in multi:
            unsplit_utterance = " ".join(lemma_utterance)
            if m in unsplit_utterance:
                intersection.add(m)
        cnt += len(intersection)
        if cnt > 0:
            return 1
    return 0

In [11]:
from torch.distributions import Categorical
from torch.autograd import Variable
device = "cuda" if torch.cuda.is_available() else "cpu"
mname = "facebook/blenderbot-400M-distill"
tokenizer = BlenderbotTokenizer.from_pretrained(mname)
model = BlenderbotForConditionalGeneration.from_pretrained(mname).to(device)
target_model = BlenderbotForConditionalGeneration.from_pretrained(mname).to(device)

Downloading:   0%|          | 0.00/124k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/61.4k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.54k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/696M [00:00<?, ?B/s]

In [12]:
!pip install undecorated

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
from random import shuffle
import sys
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
import torch

# gpt2, microsoft/DialoGPT-medium
model_id = "gpt2"
gpt_model = GPT2LMHeadModel.from_pretrained(model_id).to(device)
gpt_tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

nll = 0
def perplexity(seq):
    p = tokenizer.decode(seq, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    input_ids = gpt_tokenizer(p, return_tensors="pt").input_ids.to(device)
    with torch.no_grad():
        outputs = gpt_model(input_ids, labels=input_ids)
        nll = outputs[0].mean().item()
    ppl = torch.exp(torch.tensor(nll)).item()
    return ppl

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

KeyboardInterrupt: ignored

In [16]:
num_epoch = len(dataset)
gamma = 0.9
hit_score = 0.5
not_hit_score = -5

In [15]:
sd_persona = 'your persona: symphony and musical are relaxs to me.\nreading fictions and watching cartoons are my entertainments on airplane.'

In [23]:
import tqdm
from torch.nn import functional as F
from undecorated import undecorated
from types import MethodType
from torch.distributions import Categorical
from transformers import AdamW
generate_with_grad = undecorated(model.generate)
model.generate_with_grad = MethodType(generate_with_grad, model)
optimizer = AdamW(model.get_encoder().parameters(), lr=0.00002, correct_bias=False)
hit_record = []
cnt = 0
for index, context in enumerate(tqdm.tqdm(dataset["context"])):
    reward = 0
    train_loss = 0
    dialog = []
    prob_list = []
    perplexity_list = []
    print('now in epoch : {} / {}'.format(index + 1, num_epoch))
    hit_round = -1
    # train model
    model.train()
    optimizer.zero_grad()
    for i in range(6):
        inputs = tokenizer(
                    ["</s> <s>".join([context] + dialog if len(dialog) < 3 else dialog[-3:])], return_tensors="pt", truncation=True
                 ).to(device)
        reply_ids = target_model.generate(**inputs, do_sample=True, top_p=0.8)
        #print(reply_ids.shape)
        text = tokenizer.batch_decode(
                    reply_ids, skip_special_tokens=True
               )[0].strip()
        print('simulator: ', text)
        dialog.append(text)
        if hit(text) and i > 0:
            cnt += 1
            hit_round = i
            break
        if i == 5:
            break
        inputs = tokenizer(
                    ["</s> <s>".join([sd_persona] + dialog[-3:])], return_tensors="pt", truncation=True
                 ).to(device)
        #print(inputs)
        output = model.generate_with_grad(**inputs, num_beams=1, do_sample=True, return_dict_in_generate=True, output_scores=True)
        # add loss 
        #o_input = tokenizer(
        #            ["</s> <s>".join(dialog[-3:])], return_tensors="pt", truncation=True
        #         ).to(device)
        label = target_model.generate(**inputs, do_sample=True, top_p=0.8)
        loss = model(input_ids=inputs['input_ids'], labels=label).loss
        train_loss += loss
        seq = output.sequences
        #print('the shape of sequence : {}'.format(seq.shape))
        #print(seq.shape)
        text = tokenizer.batch_decode(
                    seq, skip_special_tokens=True
               )[0].strip()
        dialog.append(text)
        print('bot: ', text)
        score = torch.stack(output.scores).squeeze(1)
        # print('the shape of score : {}'.format(score.shape))
        # print(score.shape)
        # call probibility
        s = Categorical(logits=score)
        prob = s.log_prob(seq[0][1:])
        #print(prob.shape)
        prod_prob = torch.sum(prob)
        prob_list.append(prod_prob)
        print('the prob is : {}'.format(prod_prob))
        #perplex = perplexity(seq[0]) / 100
        #print('the perplexity is : {}'.format(perplex))
        #perplexity_list.append(perplex)
        
    for idx, prod in enumerate(prob_list):
        if not hit_round == -1:
            hit_r = hit_score * prod * (gamma ** (hit_round - idx - 1))
            print('The hit reward of round {} is : {}'.format(idx, hit_r))
            reward += hit_r
        #for i in range(idx, len(perplexity_list) - 1):
        #    reward -= prob_list[i] * perplexity_list[i] * (gamma ** (i - idx))
        else:
            hit_r = not_hit_score * prod * (gamma ** (hit_round - idx - 1))
            print('The not-hit reward of round {} is : {}'.format(idx, hit_r))
            reward += hit_r

    if not hit_round == -1:
        print('Hit!! The reward is : {}'.format(reward.item()))
        hit_record.append(1)
    else:
        #print('Not Hit!! The reward is : {}'.format(reward.item()))
        print('Not Hit!! The reward is : {}'.format(reward.item()))
        hit_record.append(0)
    reward *= -1
    train_loss += reward * 0.05
    print('The loss in round {} is {}'.format(index + 1, loss.item()))
    print('the hit rate now is : {} / {}'.format(cnt, len(dataset)))
    train_loss.backward()
    optimizer.step()

torch.save(model.state_dict(), f"./drive/MyDrive/adl-final/model/hit12_new_persona_400.ckpt")

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
  0%|          | 0/200 [00:00<?, ?it/s]

now in epoch : 1 / 1000
simulator:  I agree. I prefer headphones because I can listen to music without disturbing other people.
bot:  I do not listen to those headphones. What kind do you like? I like a lot of jazz music.
the prob is : -36.82986068725586
simulator:  I do the same thing. I don't like headphones when I'm exercising.
bot:  Yes, I would hate to get stuck in the ear while using headphones too. There are plenty of interesting things to listen to on the headphones as well. Have you tried listening to any music on your iPod?
the prob is : -76.2430419921875
simulator:  I have not, but I have been thinking about it. Do you have any recommendations?
bot:  I do not listen to much music on my iPad but I love jazz and pop music. I prefer Katy Perry. What kind of music music do you like?
the prob is : -52.78277587890625
simulator:  I like all kinds of music, but lately I've been listening to a lot of classical music.
The hit reward of round 0 is : -14.916093826293945
The hit reward o

  0%|          | 1/200 [00:06<21:12,  6.40s/it]

now in epoch : 2 / 1000
simulator:  I was. It was my first graduation and it was the best day of my life.
bot:  That's great to hear. I bet it was an exciting and interesting time. What kind of music do you like to listen to?
the prob is : -27.373504638671875
simulator:  I like all kinds of music, but my favorite genre is rock. What about you?
The hit reward of round 0 is : -13.686752319335938
Hit!! The reward is : -13.686752319335938
The loss in round 2 is 0.9847303032875061
the hit rate now is : 2 / 200


  1%|          | 2/200 [00:08<12:27,  3.77s/it]

...

 99%|█████████▉| 198/200 [13:19<00:07,  3.94s/it]

now in epoch : 199 / 1000
simulator:  I live in the northeastern part of the United States. It is very cold here.
bot:  I love the weather in the united states! It is the best. What do you like about living there?
the prob is : -32.17337417602539
simulator:  I love the cold. I'm a Viking, so I'm used to it.
bot:  I love the outdoors.  We have an indoor pig reseraurus so we go to the beach all the time.  Have you been to the ocean?
the prob is : -68.19601440429688
simulator:  No, I haven't, but I would love to go one day. Do you go often?
bot:  Not as much as I'd like. I have a lot of free time since I work nights and not enough time to do fun stuff like that.
the prob is : -50.64149856567383
simulator:  I don't get to go as often as I would like either. It's hard to get away from work.
bot:  Have you been with your husband for long now? I hope to find someone when I leave for college.
the prob is : -48.02534484863281
simulator:  I have been married for 10 years. We have been through a 

100%|█████████▉| 199/200 [13:28<00:05,  5.50s/it]

now in epoch : 200 / 1000
simulator:  I don't know. I guess she just doesn't like me. I love her though.
bot:  Not many people like her. I thought she was one of the coolest actors ever.
the prob is : -34.292083740234375
simulator:  I am sorry to hear that. I hope you are able to work things out with her.
bot:  I don't think it's a very good idea to ask people to hang out with me. Do you live in a big city?
the prob is : -48.228660583496094
simulator:  I do, I live in Los Angeles. It is a great place to be.
bot:  I do not live in the city. I'm too small to be able to see many amazing things close by.
the prob is : -39.41066360473633
simulator:  I am sorry to hear that. I am sure there are some great things to do in the suburbs though.
bot:  When I was younger, I used to love to play video games. Are you into any musical genres?
the prob is : -28.80874252319336
simulator:  I am not really into music, but I do enjoy going to concerts. What kind of music do you like?
The hit reward of rou

100%|██████████| 200/200 [13:34<00:00,  4.07s/it]


In [24]:
print('The hit ratio of the first 100 is : {}'.format(sum(hit_record[:100]) / 100))
print('The hit ratio of the last 100 is : {}'.format(sum(hit_record[100:]) / 100))

The hit ratio of the first 100 is : 0.9
The hit ratio of the last 100 is : 0.94


# Test

In [25]:
if __name__ == "__main__":
    random.seed(26)
    import torch
    total_count = 0
    device = "cuda" if torch.cuda.is_available() else "cpu"

    mname = "facebook/blenderbot-400M-distill"
    simulator = BlenderbotForConditionalGeneration.from_pretrained(mname).to(device)
    simulator_tokenizer = BlenderbotTokenizer.from_pretrained(mname)

    # load your bot
    model.load_state_dict(torch.load('/content/drive/MyDrive/adl-final/model/hit5-5_new_short_persona_400.ckpt'))
    bot_tokenizer = BlenderbotTokenizer.from_pretrained(mname)
    dataset = load_dataset("blended_skill_talk", split='test')
    dataset = dataset.map(
        preprocess,
        remove_columns=[
            "free_messages",
            "guided_messages",
            "suggestions",
            "personas",
            "additional_context",
            "previous_utterance",
        ],
    )
    if False:
        for _ in range(args.num_chats):
            dialog = ["hi"]
            while True:
                inputs = simulator_tokenizer(
                    ["</s> <s>".join(dialog[-3:])], return_tensors="pt", truncation=True
                ).to(device)
                reply_ids = simulator.generate(**inputs, do_sample=True, top_p=0.8)
                text = simulator_tokenizer.batch_decode(
                    reply_ids, skip_special_tokens=True
                )[0].strip()
                dialog.append(text)
                print(f"\033[0;32;49m {'simulator: ': ^11}{text} \033[0;0m")

                text = input(f"\033[0;33;49m {'you: ': ^11}")
                dialog.append(text)
                if text in ["stop", "exit"]:
                    break
            if text == "stop":
                break
            print()
    else:
        output = []
        test_set = test_set.select(random.sample(range(len(test_set)), 980))
        for index, context in enumerate(tqdm.tqdm(test_set["context"])):
            dialog = []
            cnt = 0
            for _ in range(5):
                inputs = simulator_tokenizer(
                    [
                        "</s> <s>".join(
                            ([context] + dialog if len(dialog) < 3 else dialog[-3:])
                        )
                    ],
                    return_tensors="pt",
                    truncation=True,
                ).to(device)
                reply_ids = simulator.generate(**inputs)
                text = simulator_tokenizer.batch_decode(
                    reply_ids, skip_special_tokens=True
                )[0].strip()
                dialog.append(text)
                if hit(text) and _ > 0:
                    cnt += 1

                print(f"\033[0;32;49m {'simulator: ': ^11}{text} \033[0;0m")

                # you might need to change this line due to the model you use
                inputs = bot_tokenizer(
                    ["</s> <s>".join([sd_persona] + dialog[-3:])], return_tensors="pt", truncation=True
                ).to(device)
                reply_ids = model.generate(**inputs, num_beams=1)
                text = bot_tokenizer.batch_decode(reply_ids, skip_special_tokens=True)[0].strip()
                dialog.append(text)
                print(f"\033[0;33;49m {'bot: ': ^11}{text} \033[0;0m")

            inputs = simulator_tokenizer(
                    [
                        "</s> <s>".join(
                            ([context] + dialog if len(dialog) < 3 else dialog[-3:])
                        )
                    ],
                    return_tensors="pt",
                    truncation=True,
                ).to(device)
            reply_ids = simulator.generate(**inputs)
            text = simulator_tokenizer.batch_decode(
                    reply_ids, skip_special_tokens=True
            )[0].strip()
            dialog.append(text)
            if hit(text):
                cnt += 1
        
            print(f"\033[0;32;49m {'simulator: ': ^11}{text} \033[0;0m")
            output.append(dialog)
            if cnt > 0:
                total_count += 1
            print('the hit rate now is : {} / {}'.format(total_count, index + 1))
        with open('/content/drive/MyDrive/adl-final/output.jsonl', "w") as f:
            for idx, dialog in enumerate(output):
                f.write(json.dumps({"id": idx, "dialog": dialog}) + "\n")

Using custom data configuration default
Reusing dataset blended_skill_talk (/root/.cache/huggingface/datasets/blended_skill_talk/default/1.0.0/bded69fdeee98ed8bba2ef088ac9dfd74e9ad0b95b1de5d51e333cee6f6261aa)
Loading cached processed dataset at /root/.cache/huggingface/datasets/blended_skill_talk/default/1.0.0/bded69fdeee98ed8bba2ef088ac9dfd74e9ad0b95b1de5d51e333cee6f6261aa/cache-5dbd867f17729d1e.arrow
  0%|          | 0/980 [00:00<?, ?it/s]

 simulator: What do you do for a living? I'm a weight lifter, so I eat a lot of meat. 
    bot:    I work at a grocery store. I love meat too. What kind of meat do you like? 
 simulator: I don't eat meat, but I do enjoy a good steak every once in a while. 
    bot:    I like steak too. I also like to eat it. I like to grill it. 
 simulator: I like grilling as well, but it's hard to find a good grill where I live. 
    bot:    I like grilled steak as well. I love it when it is grilled. 
 simulator: I like it grilled too, but I prefer it pan-fried or broiled. 
    bot:    I like it too. I like to grill it on the grill. It is very versatile. 
 simulator: I like mine grilled as well, but sometimes I like it pan fried too. 
    bot:    I like pan fries too. They are very tasty. Do you like to eat them with ketchup? 


  0%|          | 1/980 [00:03<54:51,  3.36s/it]

 simulator: Ketchup is a great condiment, but I prefer mayonnaise on my fries. 
the hit rate now is : 1 / 1
 simulator: That's true, but I like it. I read a lot of Stephen King books. 
    bot:    I love Stephen Kings books. I also like to read Stephen king. What is your favorite book of his? 
 simulator: I really like The Shining. It's one of my favorites. What about you? 
    bot:    I love Stephan King's books too. I like his short stories. Do you have a favorite? 
 simulator: I really like the Dark Tower series by Stephen King. Have you read any of his books? 
    bot:    I have read a few of his short story collections. I really enjoyed them. Do they have any of your favorites? 
 simulator: I really liked "Shawshank Redemption" and "The Shining" 
    bot:    I have read that one. It was a great book. It's one of his most popular. 
 simulator: I know, it's a classic! It was published in 1974 by Stephen King. 
    bot:    I didn't know that. I'll have to check it out. What's your fa

  0%|          | 2/980 [00:06<55:01,  3.38s/it]

...

 simulator: I like all kinds, but I especially like paintings and sculptures. What about you? 
the hit rate now is : 912 / 978
 simulator: I love to bake, but I'm not very good at it. Do you bake a lot? 
    bot:    I do. I love to make cakes and cookies. What do you like to bak? 
 simulator: I love baking cookies and brownies. I like to experiment with different ingredients. 
    bot:    I love brownies too. I also like to make cookies with peanut butter. 
 simulator: I have never tried that. I will have to give it a try. Do you have a favorite type of cookie? 
    bot:    I do. I love the chocolate chip cookies. What about you? Do you like cookies? 
 simulator: I love oreo cookies! They are my favorite. I like to bake them in the oven. 
    bot:    I love to baked too. I also like to eat them. Do your friends like to share? 
 simulator: I don't have many friends so I bake for them a lot. Do you have any hobbies? 
    bot:    I do not have any friends either. I do like to play video g

100%|█████████▉| 979/980 [55:58<00:03,  3.43s/it]

 simulator: Video games are fun. I like to bake and watch movies. What kind of video games do you like? 
the hit rate now is : 913 / 979
 simulator: That sounds lovely. What kind of flowers do you have in your garden? 
    bot:    I have a few different kinds of flowered plants. I love to garden. What about you? 
 simulator: I don't have any flowers in my garden, but I do enjoy gardening. 
    bot:    I love gardning too. I like to garden as well. What do you like to grow in your yard? 
 simulator: I grow tomatoes, cucumbers, peppers, and onions. 
    bot:    I love tomatos. I also like tomatoe. I love them on my pizza. 
 simulator: Tomatoes are my favorite vegetable. I grow them in my garden every year. 
    bot:    I love tomato too. I like tomatoes on my sandwiches. 
 simulator: Tomatoes have been around for a long time. They were first cultivated in Central Asia. 
    bot:    I love tomatos. I also like tomotoes on sandwiched. 


100%|██████████| 980/980 [56:02<00:00,  3.43s/it]

 simulator: Tomatoes are so versatile. They can be used in so many different ways. 
the hit rate now is : 914 / 980


# Submission

In [2]:
%cd /content/drive/MyDrive/adl-final/team_33
!ls

/content/drive/MyDrive/adl-final/team_33
output.jsonl  requirements.txt	run.sh	simulator.py


In [3]:
!bash run.sh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 8.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
--2022-06-13 14:46:13--  https://www.dropbox.com/s/o127npoxeh56g1u/RL_model_longperson_1000.ckpt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/o127npoxeh56g1u/RL_model_longperson_1000.ckpt [following]
--2022-06-13 14:46:14--  https://www.dropbox.com/s/dl/o127npoxeh56g1u/RL_model_longperson_1000.ckpt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf50e49dcc0eed84

In [4]:
!ls

best_model.ckpt  keywords.json	requirements.txt  simulator.py
hit.py		 output.jsonl	run.sh


In [5]:
!python3 hit.py

100% 980/980 [00:05<00:00, 173.98it/s]
total dialogs: 980
hit count by service:
	service: restaurant
		bacon 3
		egg 2
		beverage 1
		cook 22
		food 48
		serve 2
		cuisine 2
		pasta 12
		restaurant 16
		dish 3
		vegetable 8
		meat 6
		pizza 4
		pie 3
		fish 21
		eat 34
		burger 1
		pork 1
		dessert 1
		cake 5
		grill 3
		steak 6
		spaghetti 1
		season 2
		drink 3
		treat 5
		soup 2
		sushi 1
		sandwich 1
		chicken 1
		fry 3
		salmon 1
		ice cream 1
		barbecue 2
	service: hotel
		breakfast 3
		travel 76
		vacation 91
		hotel 13
		trip 8
		destination 10
		relax 27
		sleep 4
		stay 21
		resort 3
		tourist 4
		rest 2
	service: movie
		horror 5
		fiction 11
		movie 69
		fantasy 9
		television 2
		theater 2
		star 1
		direct 2
		end 6
		comedy 4
		thriller 4
		show 9
		character 1
		scene 1
		tv 3
		musical 3
		adventure 2
		documentary 1
		picture 1
	service: song
		music 147
		jazz 10
		band 54
		classical 16
		guitar 4
		rock 26
		sing 3
		concert 4
		singer 3
		piano 3
		song 14
		rhyth